In [0]:
#uploading file
from google.colab import files
uploaded = files.upload()

In [0]:
from google.colab import drive       #mouting google drive
drive.mount('/content/gdrive')

In [0]:
import os

In [0]:
os.chdir('/content/gdrive/My Drive')

In [0]:
#reading file
f = open("stemming-assignment.txt", "r")
print(f.read()) 
data=list(f.read())

In [0]:
#making file to list
li= [line.rstrip('\n') for line in open("stemming-assignment.txt")]


In [0]:
len(li)

In [0]:
li[1]

In [0]:
import os
cwd = os.getcwd()
cwd

# Finding Cluster

In [0]:
#calculating distance by formula
arr = [[0]*23531 for _ in range(23531)]

In [0]:
for i in range(len(li)):                        #finding distance
  if(i%1000==0):
    print(i)
  for j in range(len(li)):
    distance=0
    d1=list(li[i])
    d2=list(li[j])
    x=0
    y=0
    max_dis=max(len(d1),len(d2))
    while x!=len(d1) and y!=len(d2):
      if(d1[x]==d2[y]):
        distance+=1
        x+=1
        y+=1
      else:
        break
    arr[i][j]=distance/max_dis

In [0]:
import numpy as np              #save arr  
np.save('project_arr.npy', arr)

In [0]:
from numpy import load            #load arr
arr=load('/content/gdrive/My Drive/project_arr.npy')

In [0]:
cluster_word=[]                 #making cluster
for i in range(len(li)):
  temp=[]
  for j in range(len(li)):
    if(arr[i][j]>0.6):
      temp.append(li[j])
  cluster_word.append(temp)

In [0]:
fin_cul=[]                      #removing duplicate
for i in range(len(cluster_word)):
  cluster_word[i].sort()
  if cluster_word[i] not in fin_cul:
    fin_cul.append(cluster_word[i])

In [0]:
fin_cul[:50]

In [0]:
def commonPrefixUtil(str1, str2):          #function to find common prefix
	n1 = len(str1) 
	n2 = len(str2) 
	
	result = ""  
	j = 0
	i = 0
	while(i <= n1 - 1 and j <= n2 - 1): 
		if (str1[i] != str2[j]): 
			break
		result += (str1[i]) 
		
		i += 1
		j += 1

	return (result)  
def commonPrefix(arr, n):  
	arr.sort(reverse = False) 
	return commonPrefixUtil(arr[0], arr[n - 1])  
	

In [0]:
lp=[]                      #store common prefix
for i in range(len(fin_cul)):
  arr = fin_cul[i]
  n = len(arr) 
  lp.append(commonPrefix(arr, n)) 

In [0]:
len(lp)

In [0]:
temp_suffix=[]             #find temp suffix
for i in range(len(fin_cul)):
  temp=[]
  n=len(lp[i])
  for j in range(len(fin_cul[i])):
    if(n==len(fin_cul[i][j])):
      temp.append("")
    else:
      temp.append(fin_cul[i][j][n:])
  temp_suffix.append(temp)


In [0]:
len(temp_suffix[0])

In [0]:
max_len_word=0
for i in range(len(lp)):
    max_len_word=max(max_len_word,len(lp[i]))

In [0]:
max_len_word          #max length of word 

In [0]:
max_len_suffix         #max length of suffix

#Training of model

In [0]:
fin_prob=[]              #max entropy classifier
for x in range(1,28):
  temp_prob=[]
  for y in range(0,10):
    t1=0
    t2=0
    for i in range(len(fin_cul)):
      for j in range(len(fin_cul[i])):
        if(len(fin_cul[i][j])==x):
          t1+=1
          if(len(temp_suffix[i][j])==y):
            t2+=1
    if(t1!=0):
      temp_prob.append(t2/t1)
    else:
      temp_prob.append(0.0)
  fin_prob.append(temp_prob)

In [0]:
len(fin_prob)

In [0]:
input="lips"

In [0]:
L=len(input)

In [0]:
m=0
ans=0
for i in range(len(fin_prob[L-1])):
  if(m<fin_prob[L-1][i]):
    m=max(m,fin_prob[L-1][i])
    ans=i

In [0]:
ans

In [0]:
x=len(input)-ans

In [0]:
input[0:x]

In [0]:
len(li)

In [0]:
dict_f1={}                     #word with its suffix length to stripped off
for j in range(len(li)):
  L=len(li[j])
  m=0
  ans=0
  for i in range(len(fin_prob[L-1])):
    if(m<fin_prob[L-1][i]):
      m=max(m,fin_prob[L-1][i])
      ans=i
  dict_f1[li[j]]=ans

In [0]:
pip install patool

In [0]:
import patoolib          #exctrating files
patoolib.extract_archive("TELEGRAPH_UTF8.rar", outdir='/content/gdrive/My Drive')

In [0]:
import nltk
nltk.download('wordnet')


In [0]:
import nltk
nltk.download('stopwords')

#Trained Model

In [0]:
def hps(x):                           #work as trained model
  L=len(x)
  m=0
  ans=0
  for i in range(len(fin_prob[L-1])):
    if(m<fin_prob[L-1][i]):
      m=max(m,fin_prob[L-1][i])
      ans=i
  temp=L-ans
  return x[0:temp]

In [0]:
hps('lips')

#Pre processing of docs file

In [0]:
import re                          #function of contradiction
contractions_dict = {
'didn\'t': 'did not',
'don\'t': 'do not',
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(s, contractions_dict=contractions_dict):  #contraction function
    def replace(match):                                              
         return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)                 

In [0]:
from nltk.stem import PorterStemmer                    #porter stemmer import
from nltk.tokenize import word_tokenize                #assign to ps
ps = PorterStemmer()
import pandas as pd                                    #import pandas
import os                                              #import operating system library
import xml.etree.ElementTree as ET                     #import XML tree library
import re                                              #import regex
from nltk.stem.wordnet import WordNetLemmatizer        #import wordnet for lematiation
from xml.etree.ElementTree import tostring             #to convert into llist
from nltk.corpus import stopwords                      #importing corpus for stopwords
lmtzr=WordNetLemmatizer()                              #calling lemma function
new_titles=[]                                          #empty list
dataframe=[]                                           #storing final list after preprocessing
filename=[]
c=0
stop_words = stopwords.words('english')                #english stop word    
location = "/content/gdrive/My Drive/TELEGRAPH_UTF8"   #cahning the dir
for r, d, f in os.walk(location):                      #reading all files in dir
    for item in f:
        if'.utf8' in item:
            #try:
              tree = ET.parse(os.path.join(r, item))              #parsing file with in tree 
              root = tree.getroot()                               #finding root
              filename.append(root[0].text)
              t = str(tostring(root))                             #converting into the string     
              t=t.replace('\\n',' ')                              #replacing slash n with space
              t=re.sub(' +', ' ', t)                              #print(t)#adujsting space with regex
              titles = re.findall('<TEXT>(.*)</TEXT>',t)          #finding all the text betwwen TEXT tag
              titles=str(titles)                                  #convert into string   
              titles= re.sub(r'[^\w\s]','',titles)                #remove puncation
              new_titles=titles.split(" ")                        #making it into list
              new_titles=[item.lower() for item in new_titles]    #conver into lower case
              new_titles=[word for word in new_titles if word not in stop_words]#remove stop word
              new_titles=map(expand_contractions,new_titles)      #remove contraction
              temp=[]
              for x in new_titles:
                  if(x==ps.stem(x)):
                      temp.append(x)
                  else:
                    try:
                      temp.append(hps(x))
                    except:
                      temp.append(x)
              dataframe.append(str(temp))                   #making final list
              print(c)
              c+=1                                                #just to see that all dataframe working or not 
              if c%1000==0:
                  print(c)
            #except:
                  #continue;                                                   

In [0]:
len(filename)

In [0]:
len(dataframe)

#Tf-idf on docs


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer   #importing tfidf from sklearn library
v = TfidfVectorizer(use_idf=True)                                      #calling method
x = v.fit_transform(dataframe)                                #fit the data and apply it on string

In [0]:
x.shape                                   #printing shape of x

In [0]:
x=x.T

In [0]:
from scipy.sparse.linalg import svds,eigs
import numpy as np
U, s, Vh = svds(x,k=350)                           #calculating svd as k=350

In [0]:
sa=np.diag(s)                                    #s is vector matrix making it squre with digonal
sa=np.linalg.inv(sa)                             #taking invers of s

In [0]:
import numpy as np
vh_tran=np.matrix.transpose(Vh)                 #taking transpose if v

#Preporcessing of query

In [0]:
import pandas as pd                                    #import pandas
import os                                              #import operating system library
import lxml.etree as ET                                #import XML tree library
import re                                              #import regex
from nltk.stem.wordnet import WordNetLemmatizer        #import wordnet for lematiation
from xml.etree.ElementTree import tostring             #to convert into llist
from nltk.corpus import stopwords                      #importing corpus for stopwords
lmtzr=WordNetLemmatizer()                              #calling lemma function
new_titles=[]                                          #temp empty list
dataframe_new=[]                                       #final list for query matrix
doc=[]
stop_words = stopwords.words('english')                #english stop word    
tree = ET.parse('en.topics.76-125.2010.txt')           #parsing file with in tree 
root = tree.getroot() 
t = str(tostring(root))                                #same preprocess as above
t=t.replace('\\n',' ') 
t=re.sub(' +', ' ', t) 
for i in root.findall('top'):
    titles = i.find('desc').text
    t=list(filter(lambda a: a not in ['*','/','+','-','.','?',',','\n',':',';','!','(',')'], list(titles)))
    sw = stopwords.words("english")
    s="".join(t).lower()
    d=[]
    for j in s.split():
        if j not in sw:
            d.append(j)
    dataframe_new.append(" ".join(d))
    doc.append(i.find('num').text)
                                                 

In [0]:
for i in range(len(dataframe_new)):
  dataframe_new[i]=dataframe_new[i].split(" ")

In [0]:
new_data=[]
for i in range(len(dataframe_new)):
  temp=[]
  for j in range(len(dataframe_new[i])):
    if(dataframe_new[i][j]==ps.stem(dataframe_new[i][j])):
      temp.append(dataframe_new[i][j])
    else:
      try:
        temp.append(hps(dataframe_new[i][j]))
      except:
        temp.append(dataframe_new[i][j])
  new_data.append(temp)

In [0]:
que=[]
for i in range(0,50):
  listToStr=[]
  listToStr = ' '.join([str(elem) for elem in new_data[i]])
  que.append(listToStr)

# tf-idf on query

In [0]:
x1=v.transform(que)
features=v.get_feature_names()                             #getting features name

In [0]:
x1=x1.toarray()

In [0]:
import numpy as np
temp=np.matmul(x1,U)
X=np.matmul(temp,sa)

# Finding cosine similarity

In [0]:
from sklearn.metrics.pairwise import cosine_similarity   #finding cosine similarity between query exapnsion and doc vector
dictf={}
x=[x for x in range(76,126)]
for i in range(0,50):
  temp=[]
  for j in range(0,125581):
    st=cosine_similarity(X[i].reshape(1,350),vh_tran[j].reshape(1,350))
    temp.append(st)
  key=x[i]
  print(key)
  dictf.update({key : temp})
    
    


In [0]:
df=pd.DataFrame.from_dict(dictf,orient='index',columns=filename)  #making dataframe with query id map it to our document

In [0]:
final_queries={}                                     #top 10 docs via sorting
indexlist=df.index.values.tolist()
for index in indexlist:
  sorted_val=df.loc[index,:].sort_values(ascending=False).index.values.tolist()
  value=','.join(sorted_val[0:10])
  final_queries.update({index:str(value)})

In [0]:
keepitsafe={}                        #making it list of dict
for i,v in final_queries.items():
  l1=[]
  l2=[] 
  l1.append(i) 
  l2 = list(v.split(","))
  keepitsafe.update({l1[0]:l2 })

In [0]:
with open("qr.txt") as file:   #finding rel docs with doc number of given file
    
    n=76
    rele={}
    rl=[]
    count=0
    for i in file:
        t=i.split()
        if int(t[0])==n:
            if int(t[-1])==1:
                count+=1
                rl.append(t[-2])
        else:
            rele[n]=rl
            rl=[]
            n+=1
            if int(t[-1])==1:
                count+=1
                rl.append(t[-2])
    rele[n]=rl
    

In [0]:
countit={}                          #actual matched documets query wise
dict_fin={}
temp=76
c_r=0
print("ID : and its relevent docs we have found and true")
for i,v in rele.items():
  for i1,v1 in keepitsafe.items():
    if(i==i1):
      l1=[]
      j=str(i)
      l1=[value for value in v1 if value in v]
      c_r+=len(l1)
      x=' '.join([str(elem) for elem in l1])
      dict_fin[temp]=l1
      temp+=1
      print(j+'  '+x)
      countit.update({i:len(l1)})

# Finding scores

In [0]:
c_r                #no of rel docs

In [0]:
x=c_r/653          #recall

In [0]:
x

In [0]:
y=c_r/500         #precsion

In [0]:
y

In [0]:
f1=2*(x*y/(x+y))       #f1 score